# **Churned Customer Study Notebook**

## Objectives

* Write here your notebook objective, for example, "Fetch data from Kaggle and save as raw data", or "engineer features for modelling"

## Inputs

* Write here which data or information you need to run the notebook 

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here.







## Objectives

*   Answer business requirement 1: 
    * The client is interested to understand the patterns from the customer base, so the client can learn the most relevant variables that are correlated to a churned customer.

## Inputs

* outputs/datasets/collection/TelcoCustomerChurn.csv

## Outputs

* generate code that answers business requirement 1 and can be used to build the Streamlit App


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/data-driven-design/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspace/data-driven-design'

# Load Data

Section 1 content

we do not need to drop any column with drop

In [ ]:
# No need to do this!!!
#df = (pd.read_csv("outputs/datasets/collection/Airplane_Complete_Imputation.csv")
#    .drop(['customerID'], axis=1)
#    )

In [5]:
import pandas as pd
#df = (pd.read_csv("outputs/datasets/collection/Airplane_Complete_Imputation.csv")
#    .drop(['customerID'], axis=1)
#    )
df = pd.read_csv("/workspace/data-driven-design/outputs/datasets/collection/airplane_performance_study.csv")
df.head(3)



,Model,Company,Engine Type,Multi Engine,TP mods,THR,SHP,Length,Height,Wing Span,...,Vstall,Hmax,Hmax (One),ROC,ROC (One),Vlo,Slo,Vl,Sl,Range
0,15 AC Sedan,Aeronca,Piston,0,False,NaN,145.0,25.250000,10.250000,37.416667,...,46.0,13000.0,13000.0,450.0,450.0,900.0,391.970247,1300.0,257.745075,370.0
1,11 CC Super Chief,Aeronca,Piston,0,False,NaN,85.0,20.583333,8.750000,36.083333,...,44.0,12300.0,12300.0,600.0,600.0,720.0,26.247647,800.0,225.324824,190.0
2,7 CCM Champ,Aeronca,Piston,0,False,NaN,90.0,21.416667,8.583333,35.000000,...,37.0,16000.0,16000.0,650.0,650.0,475.0,363.139711,850.0,585.751317,210.0


---

# Data Exploration

We are interested to get more familiar with the dataset, check variable type and distribution, missing levels and what these variables mean in a business context

In [6]:
!pip install ydata-profiling
# Why do I need to install this when it was not necessary in the walkthrough!?
!pip install ipywidgets
# This last row is becasuse I otherwise get an error when running the next cell

Comment on below profiling report:
Model: Appear to three rows that has a duplicate model
Company
Engine Type: Highly imbalanced
Multi Engine
TP mods: Highly imbalanced
THR
SHP
Length
Height
Wing Span
Vstall
Hmax
Hmax (One)
ROC
ROC (One)
Vlo
Slo
Vl
Sl
Range

In [7]:
from ydata_profiling import ProfileReport
pandas_report = ProfileReport(df=df, minimal=True)
pandas_report.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

# Correlation Study

In [9]:
!pip install feature-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 237.9 MB/s eta 0:00:00


We are using the One Hot Encoder on the categorical features ("Engine Type", "Multi Engine") to 
"Avoids Ordinal Relationships" (One hot encoding prevents the model from assuming any ordinal relationship between the categories. For example, if you simply assigned numbers to categories (1 for Red, 2 for Green, 3 for Blue), the model might interpret Green as being "greater" than Red, which is not the case.)

In [ ]:
import pandas as pd
from feature_engine.encoding import OneHotEncoder

# Sample DataFrame
data = pd.DataFrame({
    'Color': ['Red', 'Green', 'Blue', 'Red'],
    'Size': ['S', 'M', 'L', 'S'],
    'Value': [1, 2, 3, 4]
})

# Initialize the OneHotEncoder for specific columns
encoder = OneHotEncoder(variables=['Color', 'Size'])

# Fit and transform the data
encoded_data = encoder.fit_transform(data)

print(encoded_data)


Make sure the datatype is either object or category (othervise the OneHotEncoder will not work!)

In [18]:
print(df.columns)
print(df.dtypes)

Index(['Model', 'Company', 'Engine Type', 'Multi Engine', 'TP mods', 'THR',
       'SHP', 'Length', 'Height', 'Wing Span', 'FW', 'MEW', 'AUW', 'Vmax',
       'Vcruise', 'Vstall', 'Hmax', 'Hmax (One)', 'ROC', 'ROC (One)', 'Vlo',
       'Slo', 'Vl', 'Sl', 'Range'],
      dtype='object')
Model            object
Company          object
Engine Type      object
Multi Engine      int64
TP mods            bool
THR             float64
SHP             float64
Length          float64
Height          float64
Wing Span       float64
FW              float64
MEW             float64
AUW             float64
Vmax            float64
Vcruise         float64
Vstall          float64
Hmax            float64
Hmax (One)      float64
ROC             float64
ROC (One)       float64
Vlo             float64
Slo             float64
Vl              float64
Sl              float64
Range           float64
dtype: object


In [21]:
from feature_engine.encoding import OneHotEncoder
encoder = OneHotEncoder(variables=['Engine Type'], drop_last=False)
#encoder = OneHotEncoder(variables=df.columns['Engine Type', 'Multi Engine'].to_list(), drop_last=False)
#encoder = OneHotEncoder(variables=['Engine Type', 'Multi Engine'], drop_last=False)
df_ohe = encoder.fit_transform(df)
print(df_ohe.shape)
df_ohe.head(3)

(860, 27)


,Model,Company,Multi Engine,TP mods,THR,SHP,Length,Height,Wing Span,FW,...,ROC,ROC (One),Vlo,Slo,Vl,Sl,Range,Engine Type_Piston,Engine Type_Propjet,Engine Type_Jet
0,15 AC Sedan,Aeronca,0,False,NaN,145.0,25.250000,10.250000,37.416667,241.2,...,450.0,450.0,900.0,391.970247,1300.0,257.745075,370.0,1,0,0
1,11 CC Super Chief,Aeronca,0,False,NaN,85.0,20.583333,8.750000,36.083333,100.5,...,600.0,600.0,720.0,26.247647,800.0,225.324824,190.0,1,0,0
2,7 CCM Champ,Aeronca,0,False,NaN,90.0,21.416667,8.583333,35.000000,127.3,...,650.0,650.0,475.0,363.139711,850.0,585.751317,210.0,1,0,0


In [10]:
# OLD
# from feature_engine.encoding import OneHotEncoder
encoder = OneHotEncoder(variables=df.columns[df.dtypes=='object'].to_list(), drop_last=False)
df_ohe = encoder.fit_transform(df)
print(df_ohe.shape)
df_ohe.head(3)

(860, 949)


,Multi Engine,TP mods,THR,SHP,Length,Height,Wing Span,FW,MEW,AUW,...,Company_Swift,Company_Symphony Aircraft Industries,Company_Taylorcraft Inc.,Company_Tecnam Aircraft,Company_Tradewind Turbines,Company_Twin Commander A/C,Company_Varga,Engine Type_Piston,Engine Type_Propjet,Engine Type_Jet
0,0,False,NaN,145.0,25.250000,10.250000,37.416667,241.2,1180.0,2050.0,...,0,0,0,0,0,0,0,1,0,0
1,0,False,NaN,85.0,20.583333,8.750000,36.083333,100.5,820.0,1350.0,...,0,0,0,0,0,0,0,1,0,0
2,0,False,NaN,90.0,21.416667,8.583333,35.000000,127.3,810.0,1300.0,...,0,0,0,0,0,0,0,1,0,0


---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
